## loading dataset

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("dump_truck_engine_anomaly_dataset (1).csv")
df.head()


,engine_temperature_C,oil_pressure_bar,engine_rpm,fuel_consumption_lph,vibration_mm_s,label
0,92.483571,4.870471,2079.871087,38.113444,2.162411,0
1,89.308678,5.263767,1984.926737,32.795257,2.427741,0
2,93.238443,3.940573,1811.926074,31.727204,2.103790,0
3,97.615149,4.725188,1670.612644,34.986502,2.346019,0
4,88.829233,4.239743,1939.644663,34.319262,1.553193,0


In [5]:
X = df.drop(columns=["label"])
y = df["label"]


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [7]:
# Train autoencoder ONLY on normal data
X_train = X_scaled[y == 0]


## Building the autoencoder model

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

input_dim = X_train.shape[1]

input_layer = Input(shape=(input_dim,))
encoder = Dense(32, activation="relu")(input_layer)
encoder = Dense(16, activation="relu")(encoder)

decoder = Dense(32, activation="relu")(encoder)
decoder = Dense(input_dim, activation="linear")(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse")


C:\Users\sanskar p\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## training the model

In [9]:
history = autoencoder.fit(
    X_train, X_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    shuffle=True,
    verbose=1
)


Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.6420 - val_loss: 0.6267
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5332 - val_loss: 0.5094
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4499 - val_loss: 0.4083
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3691 - val_loss: 0.3163
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2902 - val_loss: 0.2419
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2207 - val_loss: 0.1830
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1656 - val_loss: 0.1433
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1342 - val_loss: 0.1203
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1146 - val_loss: 0.1062
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0986 - val_loss: 0.0948
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0844 - val_loss: 0.0813
Epoch 12/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0718 - val_l

## calculating reconstruction error

In [10]:
X_pred = autoencoder.predict(X_scaled)
reconstruction_error = np.mean(np.square(X_scaled - X_pred), axis=1)

df["reconstruction_error"] = reconstruction_error


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


## setting anomaly threshold

In [11]:
threshold = np.percentile(
    df[df["label"] == 0]["reconstruction_error"], 95
)

print("Anomaly Threshold:", threshold)


Anomaly Threshold: 0.007362966456743521


## detecting anomalies

In [12]:
df["predicted_label"] = (df["reconstruction_error"] > threshold).astype(int)

df[["label", "predicted_label"]].head()


,label,predicted_label
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


## model evaluation

In [13]:
from sklearn.metrics import classification_report

print(classification_report(df["label"], df["predicted_label"]))


              precision    recall  f1-score   support

           0       1.00      0.95      0.97       450
           1       0.68      1.00      0.81        50

    accuracy                           0.95       500
   macro avg       0.84      0.97      0.89       500
weighted avg       0.97      0.95      0.96       500



In [14]:
feature_names = X.columns.tolist()


In [15]:
# Baseline normal sample (mean of normal data)
baseline_sample = X[y == 0].mean()


In [16]:
def interactive_anomaly_detector():
    print("\nAvailable Features:")
    for f in feature_names:
        print(" -", f)

    feature_name = input("\nEnter feature name: ").strip()

    if feature_name not in feature_names:
        print(f"\n Feature '{feature_name}' not found.")
        return

    try:
        feature_value = float(input(f"Enter value for '{feature_name}': "))
    except ValueError:
        print("\n  Invalid value. Please enter a numeric value.")
        return

    # Create baseline sample
    test_sample = baseline_sample.copy()
    test_sample[feature_name] = feature_value

    test_df = pd.DataFrame([test_sample], columns=feature_names)
    test_scaled = scaler.transform(test_df)

    reconstruction = autoencoder.predict(test_scaled)
    error = np.mean(np.square(test_scaled - reconstruction))

    print("\n Result:")
    print(f"Reconstruction Error: {error:.6f}")

    if error > threshold:
        print("🚨 Anomaly Detected")
    else:
        print("✅ Normal")


In [ ]:
interactive_anomaly_detector()



Available Features:
 - engine_temperature_C
 - oil_pressure_bar
 - engine_rpm
 - fuel_consumption_lph
 - vibration_mm_s
